In [1]:
%reload_ext autoreload
%autoreload 2

%gui qt

import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import napari
from pathlib import Path
import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import fileManagement as fm
import redCellGUI as rgui

In [3]:
# Immediate next steps for programming work:

# - database development
# - ROICaT index alignment tools


In [16]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [113]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'

# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In session: CR_Hippocannula6/2022-09-05/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Convert mpciROIs.planeIndex to mpciROIs.stackPosition, with [X,Y,Z(planeIndex)] coordinate for each ROI!
Computing red cell features... (usually takes 10-20 seconds)
C:\Users\andrew\Documents\localData\CR_Hippocannula6\2022-09-05\701


In [3]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In [3]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [13]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 3.58 s
Wall time: 512 ms


In [147]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [5]:
# database development -- simple python based API to begin with... --
import pandas as pd
import pyarrow as pa




In [16]:
dbpath = fm.codePath() / 'database'
headerColumns = []
with open(dbpath/'databaseConstruction'/'sessionHeaders.txt') as f:
    for x in f: 
        headerColumns.append(x.strip())

headerColumns

['unique identifier',
 'mouseName',
 'date',
 'session ID',
 'unique mouse ID',
 'experiment type',
 'imaging',
 'face camera',
 'vrEnvironments',
 'number of imaging planes',
 'separation of imaging planes',
 'pockels cell percentage',
 'rotation of objective',
 'rotation of head plate holder',
 'suite2p processed',
 'suite2p quality control',
 'vrExperiment registration processed',
 'red cell quality control',
 'session quality control']